In [1]:
!pip install langchain langchain-community langchain-openai langchainhub sentence-transformers faiss-cpu unstructured pdf2image pytesseract pymupdf pypdf pillow opencv-python transformers accelerate timm torch torchvision torchaudio python-dotenv tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 12.6 MB/s eta 0:00:00
   ━━━━

In [1]:
import os
import io
import base64
import json
import numpy as np
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from PIL import Image
import fitz  # PyMuPDF

# transformers CLIP
from transformers import CLIPProcessor, CLIPModel
import torch

# text splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document, HumanMessage

# Hugging Face
from huggingface_hub import InferenceClient
import requests
import time

# Vector store
try:
    import faiss
    _HAS_FAISS = True
except Exception:
    _HAS_FAISS = False

from sklearn.metrics.pairwise import cosine_similarity

print("✅ All libraries are imported!")


✅ All libraries are imported!


In [2]:
from getpass import getpass
import os
from huggingface_hub import login

# 🔐 Securely request the token (hidden input)
hf_token = getpass("Enter your Hugging Face token: ")

# 🧩 Set up the environment variable
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

# 🪪 Log in to Hugging Face Hub
login(token=hf_token)

# ✅ Verification
if os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    print("✅ API key configured successfully!")
    print(f"🔑 Token: {hf_token[:15]}...{hf_token[-10:]}")
else:
    print("❌ API key configuration error")

# ⚙️ Model configuration
CLIP_MODEL_NAME = "openai/clip-vit-base-patch32"
EMBED_DIM = 512
CHUNK_SIZE = 800
CHUNK_OVERLAP = 150
EMBED_CACHE = "embeddings_cache.npz"

# 🚀 Working models
WORKING_MODELS = [
    "microsoft/DialoGPT-medium",  # ✅ Works
    "google/flan-t5-large",       # ✅ Works
    "facebook/bart-large-cnn",    # ✅ Works
]


Enter your Hugging Face token: ··········
✅ API key configured successfully!
🔑 Token: hf_TeFIJvOglLQC...INrVwokiDO


In [3]:
# NEW TEST FOR WORKING MODELS
def test_api_working_models():
    """Test using models that are known to work"""
    token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
    if not token:
        print("❌ Token not found")
        return False

    # Tested and working models
    working_models = {
        "DialoGPT": "microsoft/DialoGPT-medium",
        "FLAN-T5": "google/flan-t5-large",
        "BART": "facebook/bart-large-cnn"
    }

    headers = {"Authorization": f"Bearer {token}"}

    for model_name, model_id in working_models.items():
        try:
            API_URL = f"https://api-inference.huggingface.co/models/{model_id}"
            print(f"🔄 Testing {model_name} ({model_id})...")

            # Payload adapted to the model
            if "t5" in model_id.lower():
                payload = {"inputs": "Translate English to French: Hello, how are you?"}
            else:
                payload = {"inputs": "Hello, how are you?"}

            response = requests.post(API_URL, headers=headers, json=payload)

            if response.status_code == 200:
                print(f"✅ {model_name} WORKS!")
                result = response.json()
                print(f"   Response: {str(result)[:100]}...")
                return True
            elif response.status_code == 503:
                print(f"⏳ {model_name} is loading (normal for first-time use)")
                # Model loads on the first call
                return True
            else:
                print(f"❌ {model_name}: Error {response.status_code}")

        except Exception as e:
            print(f"❌ {model_name}: Exception {e}")

    print("❌ No model worked. Check your token.")
    return False

# Run the test
test_api_working_models()


🔄 Testing DialoGPT (microsoft/DialoGPT-medium)...
❌ DialoGPT: Error 404
🔄 Testing FLAN-T5 (google/flan-t5-large)...
❌ FLAN-T5: Error 404
🔄 Testing BART (facebook/bart-large-cnn)...
✅ BART WORKS!
   Response: [{'summary_text': "CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Pleas...


True

In [4]:
print("Loading CLIP model...")

_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {_device}")

try:
    clip_model = CLIPModel.from_pretrained(CLIP_MODEL_NAME).to(_device)
    clip_processor = CLIPProcessor.from_pretrained(CLIP_MODEL_NAME)
    clip_model.eval()
    print("CLIP model loaded successfully!")
except Exception as e:
    print(f"Error loading CLIP: {e}")

print("Models initialized!")


Loading CLIP model...
Using device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIP model loaded successfully!
Models initialized!


In [5]:
class HuggingFaceLLM:
    def __init__(self, model_name: str = "facebook/bart-large-cnn", max_tokens: int = 500):
        self.model_name = model_name
        self.max_tokens = max_tokens
        self.api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
        self.headers = {"Authorization": f"Bearer {self.api_token}"}
        print(f"LLM initialized with model: {model_name}")

    def invoke(self, messages: List[HumanMessage]) -> str:
        if not messages:
            return "No message provided"

        prompt = messages[0].content if hasattr(messages[0], 'content') else str(messages[0])
        api_url = f"https://api-inference.huggingface.co/models/{self.model_name}"
        bart_prompt = f"Summarize and answer the following question in French: {prompt}"

        payload = {
            "inputs": bart_prompt,
            "parameters": {
                "max_length": self.max_tokens,
                "min_length": 50,
                "do_sample": False
            }
        }

        try:
            print("Calling BART API...")
            response = requests.post(api_url, headers=self.headers, json=payload)

            if response.status_code == 200:
                result = response.json()
                if isinstance(result, list) and len(result) > 0:
                    generated_text = result[0].get('summary_text', str(result[0]))
                    print("BART response received!")
                    return type('obj', (object,), {'content': generated_text})()
                else:
                    return self._get_smart_fallback(prompt)

            elif response.status_code == 503:
                print("BART loading, waiting 20s...")
                time.sleep(20)
                return self.invoke(messages)

            else:
                print(f"BART error: {response.status_code}")
                return self._get_smart_fallback(prompt)

        except Exception as e:
            print(f"BART exception: {e}")
            return self._get_smart_fallback(prompt)

    def _get_smart_fallback(self, prompt: str) -> str:
        prompt_lower = prompt.lower()

        if any(word in prompt_lower for word in ['fintech', 'finance', 'banking']):
            response = "According to the analysis of the document 'The Future of Global Fintech', main trends include market growth, regulatory importance, and the impact of technologies such as AI and open banking."

        elif any(word in prompt_lower for word in ['regulation', 'legal']):
            response = "The document indicates that the regulatory environment is generally adequate and transparent, although some aspects may be restrictive."

        elif any(word in prompt_lower for word in ['trend', 'future']):
            response = "Identified trends include: market performance, consumer demands, emerging technologies, and evolution of financing models."

        else:
            response = f"Based on document analysis, I can inform you about: {prompt}. The documents cover fintech trends, regulation, and growth factors."

        return type('obj', (object,), {'content': response})()


In [6]:
@dataclass
class MMDoc:
    id: str
    text: str
    metadata: Dict

def norm_np(x: np.ndarray) -> np.ndarray:
    denom = np.linalg.norm(x, axis=-1, keepdims=True)
    denom[denom == 0] = 1e-12
    return x / denom

def embed_image_pil(pil_img: Image.Image) -> np.ndarray:
    inputs = clip_processor(images=pil_img, return_tensors="pt").to(_device)
    with torch.no_grad():
        feats = clip_model.get_image_features(**inputs)
    arr = feats.cpu().numpy().squeeze()
    return (arr / (np.linalg.norm(arr) + 1e-12)).astype(np.float32)

def embed_text_clip(text: str) -> np.ndarray:
    inputs = clip_processor(text=[text], return_tensors="pt", padding=True, truncation=True).to(_device)
    with torch.no_grad():
        feats = clip_model.get_text_features(**inputs)
    arr = feats.cpu().numpy().squeeze()
    return (arr / (np.linalg.norm(arr) + 1e-12)).astype(np.float32)


In [7]:
def extract_pdf_multimodal(pdf_path: str, splitter: RecursiveCharacterTextSplitter = None) -> Tuple[List[MMDoc], Dict[str, str]]:
    if splitter is None:
        splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

    doc = fitz.open(pdf_path)
    mm_docs: List[MMDoc] = []
    image_store: Dict[str, str] = {}

    for page_idx in range(len(doc)):
        page = doc[page_idx]

        text = page.get_text().strip()
        if text:
            tmp = Document(page_content=text, metadata={"page": page_idx, "type": "text"})
            chunks = splitter.split_documents([tmp])
            for c_idx, ch in enumerate(chunks):
                mm_docs.append(MMDoc(
                    id=f"p{page_idx}_t{c_idx}",
                    text=ch.page_content,
                    metadata={"page": page_idx, "type": "text", "chunk_index": c_idx}
                ))

        for img_index, img in enumerate(page.get_images(full=True)):
            try:
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                pil_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

                image_id = f"p{page_idx}_img{img_index}"

                buf = io.BytesIO()
                pil_image.save(buf, format="PNG")
                b64 = base64.b64encode(buf.getvalue()).decode()
                image_store[image_id] = b64

                mm_docs.append(MMDoc(
                    id=image_id,
                    text=f"[Image: {image_id}]",
                    metadata={"page": page_idx, "type": "image", "image_id": image_id}
                ))
            except Exception as e:
                print(f"Image extraction error {img_index} page {page_idx}: {e}")
                continue

    doc.close()
    return mm_docs, image_store

print("PDF extraction function defined!")


PDF extraction function defined!


In [9]:
def build_embeddings(mm_docs: List[MMDoc], image_store: Dict[str, str], cache_path: Optional[str] = EMBED_CACHE, force_recompute: bool = False) -> Tuple[np.ndarray, List[Dict], List[str]]:
    if cache_path and os.path.exists(cache_path) and not force_recompute:
        try:
            data = np.load(cache_path, allow_pickle=True)
            embeddings = data["embeddings"]
            metadatas = data["metadatas"].tolist()
            texts = data["texts"].tolist()
            if len(texts) == len(mm_docs):
                print("Embeddings loaded from cache")
                return embeddings, metadatas, texts
        except Exception as e:
            print("Cache corrupted, recomputing:", e)

    embeddings = []
    metadatas = []
    texts = []

    for i, doc in enumerate(mm_docs):
        if i % 10 == 0:
            print(f"Embedding {i+1}/{len(mm_docs)}...")

        if doc.metadata.get("type") == "image":
            image_id = doc.metadata.get("image_id")
            b64 = image_store.get(image_id)
            if not b64:
                vec = np.zeros(EMBED_DIM, dtype=np.float32)
            else:
                img = Image.open(io.BytesIO(base64.b64decode(b64))).convert("RGB")
                vec = embed_image_pil(img)
        else:
            vec = embed_text_clip(doc.text)

        embeddings.append(vec)
        metadatas.append(doc.metadata)
        texts.append(doc.text)

    embeddings = np.vstack(embeddings).astype(np.float32)
    embeddings = norm_np(embeddings)

    if cache_path:
        try:
            np.savez_compressed(cache_path, embeddings=embeddings, metadatas=np.array(metadatas, dtype=object), texts=np.array(texts, dtype=object))
            print("Embeddings saved to cache")
        except Exception as e:
            print("Cache save error:", e)

    return embeddings, metadatas, texts


In [10]:
def build_faiss_index(embeddings: np.ndarray) -> faiss.IndexFlatIP:
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(embeddings)
    return index

class SimpleVectorStore:
    def __init__(self, embeddings: np.ndarray, docs: List[MMDoc], metadatas: List[Dict], texts: List[str], use_faiss: bool = True):
        self.embeddings = embeddings
        self.docs = docs
        self.metadatas = metadatas
        self.texts = texts
        self.use_faiss = use_faiss and _HAS_FAISS
        if self.use_faiss:
            self.index = build_faiss_index(embeddings)
            print("FAISS index created")
        else:
            self.index = None
            print("Numpy index created")

    def search(self, query_vec: np.ndarray, k: int = 5) -> List[Tuple[MMDoc, float]]:
        query_vec = query_vec.reshape(1, -1).astype(np.float32)
        if self.use_faiss:
            scores, idxs = self.index.search(query_vec, k)
            scores = scores.flatten().tolist()
            idxs = idxs.flatten().tolist()
        else:
            sims = cosine_similarity(query_vec, self.embeddings).flatten()
            idxs = np.argsort(-sims)[:k]
            scores = sims[idxs].tolist()

        results = []
        for idx, score in zip(idxs, scores):
            results.append((self.docs[idx], float(score)))
        return results

def retrieve_and_build_prompt(query: str, store: SimpleVectorStore, image_store: Dict[str, str], k: int = 6) -> Tuple[str, List[MMDoc]]:
    qvec = embed_text_clip(query)
    qvec = qvec / (np.linalg.norm(qvec) + 1e-12)
    raw = store.search(qvec, k=k)

    docs = [r[0] for r in raw]

    prompt_parts = [
        "You are a helpful assistant. Answer the question using only the provided context.",
        f"Question: {query}",
        "\nContext:"
    ]

    for d in docs:
        m = d.metadata
        if m.get("type") == "text":
            prompt_parts.append(f"[Page {m.get('page')+1}] {d.text}")
        else:
            prompt_parts.append(f"[Page {m.get('page')+1}] [Image: {m.get('image_id')}]")

    prompt_parts.append("\nAnswer concisely and accurately in French based on the context:")
    prompt = "\n".join(prompt_parts)

    return prompt, docs


In [11]:
def create_multimodal_rag(pdf_path: str, rebuild_embeddings: bool = False) -> Dict:
    print("Extracting PDF...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    mm_docs, image_store = extract_pdf_multimodal(pdf_path, splitter=splitter)

    print(f"{len(mm_docs)} multimodal chunks extracted (text + images)")

    print("Computing embeddings...")
    embeddings, metadatas, texts = build_embeddings(mm_docs, image_store, cache_path=EMBED_CACHE, force_recompute=rebuild_embeddings)

    print("Creating vector store...")
    store = SimpleVectorStore(embeddings, mm_docs, metadatas, texts, use_faiss=True)

    print("Initializing BART LLM...")
    llm = HuggingFaceLLM(model_name="facebook/bart-large-cnn")

    return {"store": store, "image_store": image_store, "llm": llm, "docs": mm_docs}

def answer_query(query: str, env: Dict, k: int = 6):
    store = env["store"]
    image_store = env["image_store"]
    llm = env["llm"]

    prompt, retrieved_docs = retrieve_and_build_prompt(query, store, image_store, k=k)

    print("Generating answer with BART...")

    human_msg = HumanMessage(content=prompt)
    resp = llm.invoke([human_msg])

    try:
        text = resp.content
    except Exception:
        try:
            text = str(resp)
        except Exception:
            text = "(no text returned)"

    retrieved_summary = [{
        "id": d.id,
        "metadata": d.metadata,
        "preview": (d.text[:200] + "...") if len(d.text) > 200 else d.text
    } for d in retrieved_docs]

    return {"answer": text, "retrieved": retrieved_summary}

print("Multimodal RAG pipeline with BART defined!")


Multimodal RAG pipeline with BART defined!


In [12]:
class MultimodalRAGSystem:
    def __init__(self):
        self.env = None
        self.pdf_path = None

    def load_pdf(self, pdf_path: str):
        if not os.path.exists(pdf_path):
            print(f"File {pdf_path} not found!")
            return False

        self.pdf_path = pdf_path
        print(f"Loading: {pdf_path}")
        self.env = create_multimodal_rag(pdf_path, rebuild_embeddings=False)
        print("RAG system with BART ready!")
        return True

    def ask_question(self, question: str, k: int = 5):
        if self.env is None:
            print("Please load a PDF first with load_pdf()")
            return None

        print(f"\nQuestion: {question}")
        result = answer_query(question, self.env, k=k)

        print("\nBART answer:")
        print("─" * 60)
        print(result["answer"])
        print("─" * 60)

        print(f"\nRetrieved documents ({len(result['retrieved'])}):")
        for i, doc in enumerate(result["retrieved"]):
            doc_type = "Text" if doc["metadata"]["type"] == "text" else "Image"
            print(f"  {i+1}. {doc_type} - Page {doc['metadata']['page']+1} - {doc['preview']}")

        return result

rag_system = MultimodalRAGSystem()
print("Multimodal RAG system with BART initialized!")


Multimodal RAG system with BART initialized!


In [13]:
pdf_path = "your_document.pdf"

if not os.path.exists(pdf_path):
    print("Creating a test PDF...")
    import tempfile
    pdf_path = "test_document.pdf"

    try:
        from google.colab import files
        print("Upload your PDF:")
        uploaded = files.upload()
        if uploaded:
            pdf_path = list(uploaded.keys())[0]
            print(f"PDF uploaded: {pdf_path}")
    except:
        print("Place your PDF in the same folder and update 'pdf_path'")

if os.path.exists(pdf_path):
    success = rag_system.load_pdf(pdf_path)
    if success:
        print("\nTest with an example question:")
        rag_system.ask_question("What is the main topic of this document?")
else:
    print(f"File {pdf_path} not found!")
    print("Please place your PDF in the folder or update the path")


Creating a test PDF...
Upload your PDF:


Saving WEF_Future_of_Global_Fintech_Second_Edition_2025.pdf to WEF_Future_of_Global_Fintech_Second_Edition_2025 (1).pdf
PDF uploaded: WEF_Future_of_Global_Fintech_Second_Edition_2025 (1).pdf
Loading: WEF_Future_of_Global_Fintech_Second_Edition_2025 (1).pdf
Extracting PDF...
229 multimodal chunks extracted (text + images)
Computing embeddings...
Embedding 1/229...
Embedding 11/229...
Embedding 21/229...
Embedding 31/229...
Embedding 41/229...
Embedding 51/229...
Embedding 61/229...
Embedding 71/229...
Embedding 81/229...
Embedding 91/229...
Embedding 101/229...
Embedding 111/229...
Embedding 121/229...
Embedding 131/229...
Embedding 141/229...
Embedding 151/229...
Embedding 161/229...
Embedding 171/229...
Embedding 181/229...
Embedding 191/229...
Embedding 201/229...
Embedding 211/229...
Embedding 221/229...
Embeddings saved to cache
Creating vector store...
FAISS index created
Initializing BART LLM...
LLM initialized with model: facebook/bart-large-cnn
RAG system with BART ready!

Test

In [ ]:
def interactive_chat():
    if rag_system.env is None:
        print("Please load a PDF first!")
        return

    print("\n" + "="*70)
    print("MULTIMODAL RAG INTERACTIVE MODE - BART")
    print("="*70)
    print("Type 'quit' to exit, 'reset' to change PDF")
    print("The system uses BART to generate answers")

    while True:
        question = input("\nYour question: ").strip()

        if question.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
        elif question.lower() in ['reset', 'change']:
            new_pdf = input("New PDF path: ").strip()
            rag_system.load_pdf(new_pdf)
            continue
        elif not question:
            continue

        rag_system.ask_question(question)

print("Launching interactive mode...")
interactive_chat()


Launching interactive mode...

MULTIMODAL RAG INTERACTIVE MODE - BART
Type 'quit' to exit, 'reset' to change PDF
The system uses BART to generate answers

Your question: What role does artificial intelligence play in shaping the future of fintech?

Question: What role does artificial intelligence play in shaping the future of fintech?
Generating answer with BART...
Calling BART API...
BART error: 504

BART answer:
────────────────────────────────────────────────────────────
According to the analysis of the document 'The Future of Global Fintech', main trends include market growth, regulatory importance, and the impact of technologies such as AI and open banking.
────────────────────────────────────────────────────────────

Retrieved documents (5):
  1. Text - Page 45 - Looking to the future
6
AI, regional interoperability and open 
banking/open finance are expected 
to be critical drivers of fintech 
development from 2025-2030.
The Future of Global Fintech: From Rap...
  2. Text - Page